# 🔄 Notebook: 01 ETL Silver Layer

This notebook handles the **Silver Layer (Data Transformation & Processing)** in the Medallion Architecture of the AI-powered claims pipeline. It focuses on **audio conversion**, **metadata extraction**, and **speech-to-text transcription** using the [OpenAI Whisper model](https://openai.com/index/whisper/).

---

## 🧱 Purpose

To convert raw audio files into a consistent format (MP3), calculate metadata (duration), and transcribe the content into structured text to support downstream AI analytics.



In [0]:
%pip install pydub mutagen openai-whisper numpy>=1.24
dbutils.library.restartPython()

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached openai_whisper-20240930-py3-none-any.whl
  Using cached more_itertools-10.6.0-py3-none-any.whl.metadata (37 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Using cached mutagen-1.47.0-py3-none-any.whl (194 kB)
Using cached more_itertools-10.6.0-py3-none-any.whl (63 kB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run "./resources/init" 

DataFrame[]

In [0]:
bronze_df = spark.table(f"{CATALOG}.{SCHEMA}.{BRONZE_TABLE}")

meta_table_name = f"{CATALOG}.{SCHEMA}.{META_TABLE}"
if spark._jsparkSession.catalog().tableExists(meta_table_name):
    processed_df = spark.table(meta_table_name).filter("processed = True")
    file_reference_df = bronze_df.join(processed_df, "file_name", "left_anti")
else:
    file_reference_df = bronze_df

if file_reference_df.isEmpty():
    dbutils.notebook.exit("✅ No new files to process. Exiting Silver Layer.")

In [0]:
from pydub import AudioSegment
import os

dbutils.fs.mkdirs(mp3_audio_path)

# Convert each file to mp3 and save to the new volume
for row in file_reference_df.collect():
    file_path = row['file_path']
    try:
        audio = AudioSegment.from_file(file_path)
        new_file_path = os.path.join(mp3_audio_path, os.path.basename(file_path).replace(os.path.splitext(file_path)[1], ".mp3"))
        audio.export(new_file_path, format="mp3")
    except Exception as e:
        print(f"⚠️ Error converting {file_path}: {e}")

In [0]:
from mutagen.mp3 import MP3
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

mp3_df = (
    spark.createDataFrame(dbutils.fs.ls(mp3_audio_path))
    .withColumn("file_path", F.expr("substring(path, 6, length(path))"))
    .withColumn("file_name", F.expr("substring(name, 1, length(name) - 4)"))
    .filter(F.col("file_name").isin([r["file_name"] for r in file_reference_df.collect()]))
)

def get_audio_duration(file_path):
    try:
        audio = MP3(file_path)
        return float(audio.info.length)
    except Exception as e:
        print(f"⚠️ Error getting duration for {file_path}: {e}")
        return None

duration_udf = F.udf(get_audio_duration, FloatType())
mp3_df = mp3_df.withColumn("audio_duration", F.round(duration_udf("file_path"), 0))

display(mp3_df)

path,name,size,modificationTime,file_path,file_name,audio_duration
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,724461,1744820609000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,91.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,865197,1744820601000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,108.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45.mp3,914349,1744820603000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45,114.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,946605,1744820605000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33,118.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,956013,1744820607000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51,119.0


In [0]:
import whisper

model = whisper.load_model("small")
print("✅ Whisper model loaded.")

def transcribe_audio(file_path: str, model: whisper.Whisper) -> str:
    try:
        result = model.transcribe(file_path)
        return result["text"]
    except Exception as e:
        print(f"⚠️ Transcription failed for {file_path}: {e}")
        return ""
      
from pyspark.sql.types import StringType
# from pyspark.sql.functions import udf

# transcribe_udf = udf(lambda path: transcribe_audio(path), StringType())

# transcribed_df = mp3_df.withColumn("transcription", transcribe_udf("file_path"))

# Collect the file paths to the driver
file_paths = mp3_df.select("file_path").rdd.flatMap(lambda x: x).collect()

# Transcribe the audio files outside of Spark
transcriptions = [transcribe_audio(file_path, model) for file_path in file_paths]

# Create a DataFrame with the transcriptions
transcriptions_df = spark.createDataFrame(zip(file_paths, transcriptions), ["file_path", "transcription"])

# Join the transcriptions back to the original DataFrame
transcribed_df = mp3_df.join(transcriptions_df, on="file_path", how="inner") \
                                 .select("file_path", "file_name", "transcription", "audio_duration")


Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 71.8MiB/s]
/local_disk0/.ephemeral_nfs/envs/pythonEnv-c14fafbc-24d1-428e-8ab8-fec9f5a35a53/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where 

✅ Whisper model loaded.


/local_disk0/.ephemeral_nfs/envs/pythonEnv-c14fafbc-24d1-428e-8ab8-fec9f5a35a53/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-c14fafbc-24d1-428e-8ab8-fec9f5a35a53/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-c14fafbc-24d1-428e-8ab8-fec9f5a35a53/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-c14fafbc-24d1-428e-8ab8-fec9f5a35a53/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  wa

In [0]:
##### For simulated data purposes, not part of final solution

from pyspark.sql.functions import expr, substring, regexp_replace, col, split, to_timestamp, concat_ws

silver_table_path = f"{CATALOG}.{SCHEMA}.{SILVER_TABLE}"

if first_run:
    # Load existing transcriptions from the simulated table
    existing_transcriptions_df = spark.table(f"{CATALOG}.{SCHEMA}.simulated_transcriptions")
    
    # Extract file name from file path
    existing_transcriptions_df = existing_transcriptions_df.withColumn("file_name", expr("substring(file_path, -39, 35)"))
    
    # Combine existing (simulated) transcriptions with new transcriptions
    combined_transcriptions_df = existing_transcriptions_df.unionByName(transcribed_df)
    
    # Extract and transform relevant columns from file name
    combined_transcriptions_df = combined_transcriptions_df.withColumn("file_name", split(col("file_path"), "/").getItem(6)) \
        .withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
        .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
        .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
        .withColumn("call_datetime", 
            to_timestamp(
                concat_ws(":", split(col("file_name"), "_").getItem(2), 
                split(col("file_name"), "_").getItem(3), 
                split(col("file_name"), "_").getItem(4))))
    
    # Display the combined DataFrame
    display(combined_transcriptions_df)
    
    # Overwrite the transcriptions_silver table with the combined DataFrame
    combined_transcriptions_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_path)

else:
    transcribed_df = transcribed_df.withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
        .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
        .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
        .withColumn("call_datetime", 
            to_timestamp(
                concat_ws(":", split(col("file_name"), "_").getItem(2), 
                split(col("file_name"), "_").getItem(3), 
                split(col("file_name"), "_").getItem(4)))
        )

    display(transcribed_df.select("file_name", "call_id", "agent_id", "call_datetime", "audio_duration", "transcription"))

    if not spark._jsparkSession.catalog().tableExists(silver_table_path):
        transcribed_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_path)
    else:
        transcribed_df.write.mode("append").saveAsTable(silver_table_path)

    print(f"✅ Transcriptions written to Silver table: {silver_table_path}")

file_path,file_name,transcription,audio_duration,call_id,agent_id,call_datetime
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/76bc767f_AGT003_2025-02-19 21:46:48.mp3,76bc767f_AGT003_2025-02-19 21:46:48,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Christopher Lopez. I was hoping to discuss my policy coverage due to a change in my income. Of course, Christopher. For security, can I confirm your date of birth and policy number? Sure, my DOB is 11th January 1971, and my policy number is VG640918. Thank you for confirming. You're calling about a change in your income and how it affects your policy. Can you tell me a bit more about your situation? Yes, I've recently experienced an income loss and I'm finding it difficult to keep up with my current premium payments. I was wondering if it's possible to lower my coverage temporarily. I understand your situation, Christopher. Let me see what options are available to you. (pause) Yes, we can discuss adjusting your policy coverage temporarily. This would involve reducing your premium payments, but it would also mean some changes to the benefits you're currently receiving. That sounds like it could work for me. What kind of changes to my benefits are we talking about? The adjustments would primarily affect your outpatient and dental coverage. You would still have access to inpatient care and emergency services, but with some limitations on the outpatient side. I can walk you through the specifics and answer any questions you have. Okay, that makes sense. What are the next steps to adjust my policy? I'll guide you through the process. I'll need to update your policy details and send you a revised policy document outlining the changes. You'll receive an email with the updated terms and a new premium amount. If you have any questions or concerns, please don't hesitate to reach out. Alright, thank you for your help. You're welcome, Christopher. Just to summarize, we've discussed adjusting your policy coverage temporarily due to your income loss. I've outlined the changes to your benefits, and I'll be sending you an updated policy document. If you need any further assistance, don't hesitate to contact us. Have a good day! Thanks, you too. Goodbye!",112.0,76bc767f,AGT003,2025-02-19T21:46:48Z
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/2ed71ef6_AGT005_2025-03-14 07:13:48.mp3,2ed71ef6_AGT005_2025-03-14 07:13:48,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Alexander Walker. I was hoping to get an update on my claim status. Of course, Alexander. For security, can I confirm your date of birth and policy number? Sure, my DOB is 31st August 1994, and my policy number is VG564705. Thank you for confirming. I've located your claim. Can you please tell me a little bit more about what's confusing you regarding your claim status? I just haven't received any updates, and I'm not sure if it's been processed yet. I understand your concern, Alexander. Let me check on the status for you... Yes, your claim is currently being reviewed. I can offer to expedite the process or provide more information on what to expect next. That would be great, thank you. You're welcome. I'm going to go ahead and expedite your claim. You should receive an update within the next 3-5 business days. Is there anything else I can assist you with today? No, that's all for now. Thank you for your help. Great! Just to summarize, I have expedited your claim, and you can expect an update within 3-5 business days. If you need further assistance, don’t hesitate to reach out. Have a wonderful day! Thanks! You too. Goodbye!",283.0,2ed71ef6,AGT005,2025-03-14T07:13:48Z
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/b95032a2_AGT001_2025-02-07 10:01:48.mp3,b95032a2_AGT001_2025-02-07 10:01:48,"H

## ✅ Output
- A clean, enriched Delta table: transcriptions_silver
- Includes transcription text, call metadata, and audio duration for each entry.